**Notes:**

- Run all instuctions below on a HOST machine
- Here we continue to work within the `experiments` branch 

# Build  CI pipeline

## Register gitlab runners

### Add credential variables

1. At the repository page go to Settings -> CI / CD -> Variables;
2. Click Expand;
3. Add variable:

```dotenv
repo_token=<access_token>
```

**Note**: remove flag `Protect variable`.



### Get register token

1. go to **Settings** -> **CI / CD** -> **Specific Runners** -> **Set up a specific Runner manually**;
2. copy  registration token from `3.Use the following registration token during setup:`;
3. define env var:

```bash
export REGISTRATION_TOKEN=<registration_token>
```

### shell runner 

```bash
gitlab-runner register \
            --non-interactive \
            -u https://gitlab.com/ \
            -r $REGISTRATION_TOKEN \
            --tag-list shell \
            --executor shell 
```

__Notes__:
* *--non-interactive* - to avoid gitlab-runner prompts;
* *-u* - CI server URL.


### docker runner

```bash
gitlab-runner register \
            --non-interactive \
            -u https://gitlab.com/ \
            -r $REGISTRATION_TOKEN \
            --tag-list docker,experiments \
            --executor docker \
            --docker-image mlrepa/cml:latest \
            --docker-disable-cache \
            --docker-volumes "/tmp:/tmp"
```

__Notes__:
* *--docker-image* - docker image which will be used by default to 
run CI job if another image will not be specified in job definition;
* *--docker-disable-cache* - avoid caching repository and data in container's volume,
volume will not be stored; it guarantees repository is fresh at every `cml` run;
if cache enabled repository can stay outdated;
* *--docker-volumes* - mount folders from host to docker container; 
it gives access to local DVC remote.

## Start runners

```bash
gitlab-runner run 
```

## Create .gitlab-ci.yml file



```yaml
variables:
  DEPLOY_IMAGE: mlrepa/sklearn_deploy_service:latest
  DEPLOY_PORT: 9000
  CML_MAGE: mlrepa/cml:latest
  PROJECT_MODEL_PATH: models/model.joblib
  DOCKER_MODEL_PATH: /home/deploy/models/model.joblib

stages:
  - build
  - test
  - experiments
```

## Create `build` job

```yaml

build:
  tags:
    - shell
  only:
    refs:
      - merge_requests
    variables:
      - $CI_MERGE_REQUEST_TARGET_BRANCH_NAME == "master"
  stage: build
  script:
    - docker pull $DEPLOY_IMAGE
    - |
      if [ ! -d "/tmp/$CI_PROJECT_NAME" ]; then
        project_url=${CI_PROJECT_URL//"https://"/}
        git clone \
            https://oauth2:$repo_token@${project_url}.git \
            "/tmp/$CI_PROJECT_NAME"
      fi
      cd /tmp/$CI_PROJECT_NAME
      git fetch
      git checkout $CI_COMMIT_REF_NAME
      git pull origin $CI_COMMIT_REF_NAME
      dvc pull

```

## Create `test` job

```yaml
test:
  tags:
    - shell
  only:
    refs:
      - merge_requests
    variables:
      - $CI_MERGE_REQUEST_TARGET_BRANCH_NAME == "master"
  stage: test
  script:
    - cd /tmp/$CI_PROJECT_NAME
    - |
      docker run \
            --name=deploy-sklearn-test \
            --rm \
            -v $(pwd)/tests:/home/deploy/tests \
            -v $(pwd)/$PROJECT_MODEL_PATH:$DOCKER_MODEL_PATH \
            -e MODEL_PATH=$DOCKER_MODEL_PATH \
                $DEPLOY_IMAGE /bin/bash -c "
                    pytest /home/deploy/tests/test_model.py
                "
```


### Add `tests/test_model.py` module

```python

import joblib
import os
import numpy as np
import pytest


@pytest.fixture(scope='module')
def model():

    model_path = os.getenv('MODEL_PATH')
    model = joblib.load(model_path)

    return model


def test_model(model):

    data = [
        {
            'sepal_length': 4.9,
            'sepal_width': 3.0,
            'petal_length': 1.7,
            'petal_width': 0.2,
            'sepal_length_to_sepal_width': 1.63333333,
            'petal_length_to_petal_width': 8.5
        }
    ]
    values = [list(d.values()) for d in data]
    np_array = np.array(values)
    result = model.predict(np_array)

    assert len(result) == 1

    for v in result:
        assert v in [0, 1, 2]
```

## Create `cml` job

```yaml
      
cml:
  tags:
    - docker,experiments
  stage: experiments
  image: mlrepa/cml:latest
  only:
    refs:
      - merge_requests
    variables:
      - $CI_MERGE_REQUEST_TARGET_BRANCH_NAME == "experiments"
  script:
    - apt-get install -y jq
    - git fetch
    - git branch --list
    - git checkout $CI_COMMIT_REF_NAME
    - git pull origin $CI_COMMIT_REF_NAME
    - dvc pull --all-commits -q

    # Report metrics
    - echo "## Metrics report for $CI_COMMIT_REF_NAME" >> report.md
    - |
      echo "## f1_score: $(cat reports/metrics.json | jq '.f1_score')" >> report.md
    - dvc metrics diff --show-md --all experiments  >> report.md
    - echo >> report.md
    - cml-publish reports/confusion_matrix.png --md >> report.md

    # Publish confusion matrix diff
    - cml-send-comment report.md
```

## Commit CI pipeline

```bash
git add .gitlab-ci.yml
git commit -m "Build CI pipeline"
```


## Trigger CI pipelines with merge events


### Trigger `cml` job

```bash 
git push origin experiments
```


### Trigger `build` and `test` jobs

Create Merge Request from `experiments` to `master`

